In [2]:
from opentrons import robot, containers, instruments
import pandas as pd
import argparse
import sys

In [3]:
# Load our files
plates = pd.read_csv('../synth2/Twist_10-26-2017_648.csv',usecols=['Plate','Well','customer_line_item_id','synthesized sequence length','Yield (ng)'])

#layout = pd.read_csv('./testing/plate-loc-resuspend-sm.csv')
#layout = layout.set_index('Position').to_dict()['Name'] # Turn into a location->name dict


In [4]:
# Configuration
SOURCE_SLOTS = ['D1','D2','D3', 'E1','B2']

plate_names = plates['Plate'].unique()

if len(plate_names) > len(SOURCE_SLOTS):
    print("Error: This build plan requires too many source plates.")
    sys.exit(1)

layout = list(zip(SOURCE_SLOTS[:len(plate_names)], plate_names))

slots = pd.Series(SOURCE_SLOTS)
columns = sorted(slots.str[0].unique())
rows = sorted(slots.str[1].unique(), reverse=True)

layout_table = pd.DataFrame(index=rows, columns=columns)
layout_table.fillna("", inplace=True)

for slot, plate in layout:
    layout_table.loc[slot[1], slot[0]] = plate

print("Please arrange the plates in the following configuration:")
print()
print(layout_table)
print()
input("Press enter to continue")

layout = dict(layout)

Please arrange the plates in the following configuration:

  B                   D E
3                        
2                        
1    pSHPs1025B525648MU  

Press enter to continue


In [7]:
# Calculate Resuspension Volumes

plates['customer_line_item_id'] = plates['customer_line_item_id'].str.strip()

amount = plates['Yield (ng)']
length = plates['synthesized sequence length']

fmoles = 40

volume = (((((amount / 1000)*(1000000))/(660*length))*1000) / fmoles) * 2

plan = pd.DataFrame({'Plate': plates['Plate'],
                     'Well': plates['Well'],
                     'Volume': volume})
plan = plan[['Plate','Well','Volume']]


In [47]:
# Configure the robot

#  Layout:
#    A     B       C      D      E
#  3 trash master  source source p10
#  2 p200  dest    source source p10
#  1 p200  dest    source source p10

robot.connect('Virtual Smoothie') #robot.get_serial_ports_list()[0])



In [9]:
dest_plates = [
    containers.load('96-flat', 'B1'),
    containers.load('96-flat', 'B2')
]

source_plates = {}
for slot, plate in layout.items():
    source_plates[plate] = containers.load('96-flat', slot)


In [21]:
# Distribute down the plates
num_reactions = len(plan)
num_rows = num_reactions // 8 + 1
all_wells = dest_plates[0].wells() + dest_plates[1].wells()

all_wells[:num_reactions]

<WellSeries: <Well A1><Well B1><Well C1><Well D1><Well E1><Well F1><Well G1><Well H1><Well A2><Well B2><Well C2><Well D2><Well E2><Well F2><Well G2><Well H2><Well A3><Well B3><Well C3><Well D3><Well E3><Well F3>>